In [1]:
import os
import json
import time
from typing import List, Dict, Any
from pathlib import Path

# Core imports
import autogen
from autogen import ConversableAgent, GroupChat, GroupChatManager

# LangChain imports
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document, HumanMessage

# ChromaDB imports
import chromadb
from chromadb.config import Settings



In [ ]:
# Azure Configuration
AZURE_CONFIG = {
    "api_key": ,
    "base_url": "",
    "api_version": "2025-01-01-preview",
    "embedding_deployment": "text-embedding-ada-002",
    "gpt_deployment": "gpt-4o"
}


In [ ]:

class InsuranceRAGSystem:
    """Enhanced Insurance RAG system with ChromaDB backend"""
    
    def __init__(self, data_folder: str = "data", persist_directory: str = "chroma_db"):
        self.data_folder = data_folder
        self.persist_directory = persist_directory
        self.collection_name = "insurance_documents"
        self.embeddings = None
        self.vectorstore = None
        self.llm = None
        self.chroma_client = None
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        self.setup_azure_clients()
        self.setup_chromadb()
        # Ensure vectorstore is always initialized
        self.create_or_load_vectorstore()
        
    def setup_azure_clients(self):
        """Initialize Azure OpenAI clients"""
        try:
            # Initialize embeddings
            self.embeddings = AzureOpenAIEmbeddings(
                azure_deployment=AZURE_CONFIG["embedding_deployment"],
                openai_api_version=AZURE_CONFIG["api_version"],
                azure_endpoint=AZURE_CONFIG["base_url"],
                openai_api_key=AZURE_CONFIG["api_key"]
            )
            
            # Initialize LLM
            self.llm = AzureChatOpenAI(
                azure_deployment=AZURE_CONFIG["gpt_deployment"],
                openai_api_version=AZURE_CONFIG["api_version"],
                azure_endpoint=AZURE_CONFIG["base_url"],
                openai_api_key=AZURE_CONFIG["api_key"],
                temperature=0.1,
                max_retries=3,
                request_timeout=60
            )
            print(" Azure OpenAI clients initialized successfully")
            
        except Exception as e:
            print(f" Error initializing Azure clients: {e}")
            raise
    
    def setup_chromadb(self):
        """Initialize ChromaDB client for persistent storage"""
        try:
            # Ensure persist directory exists
            os.makedirs(self.persist_directory, exist_ok=True)
            
            # Initialize ChromaDB client
            self.chroma_client = chromadb.PersistentClient(
                path=self.persist_directory,
                settings=Settings(
                    anonymized_telemetry=False,
                    allow_reset=True
                )
            )
            print(f" ChromaDB client initialized at: {self.persist_directory}")
            
        except Exception as e:
            print(f" Error initializing ChromaDB: {e}")
            raise
    
    def load_and_process_documents(self) -> List[Document]:
        """Load and process insurance documents"""
        try:
            documents = []
            
            # Load PDF documents
            pdf_files = list(Path(self.data_folder).glob("**/*.pdf"))
            for pdf_file in pdf_files:
                try:
                    loader = PyPDFLoader(str(pdf_file))
                    docs = loader.load_and_split()
                    documents.extend(docs)
                    print(f" Loaded PDF: {pdf_file.name}")
                except Exception as e:
                    print(f" Error loading {pdf_file}: {e}")
            
            # Load text documents
            text_files = list(Path(self.data_folder).glob("**/*.txt"))
            for text_file in text_files:
                try:
                    loader = TextLoader(str(text_file), encoding='utf-8')
                    docs = loader.load()
                    documents.extend(docs)
                    print(f" Loaded text: {text_file.name}")
                except Exception as e:
                    print(f" Error loading {text_file}: {e}")
            
            # Load Word documents
            docx_files = list(Path(self.data_folder).glob("**/*.docx"))
            for docx_file in docx_files:
                try:
                    loader = Docx2txtLoader(str(docx_file))
                    docs = loader.load()
                    documents.extend(docs)
                    print(f" Loaded DOCX: {docx_file.name}")
                except Exception as e:
                    print(f" Error loading {docx_file}: {e}")
            
            if not documents:
                print(" Creating sample insurance document")
                sample_doc = Document(
                    page_content="""
                    Comprehensive Insurance Knowledge Base:
                    
                    AUTO INSURANCE:
                    - Liability Coverage: Mandatory in most states, covers damages to others
                    - Collision Coverage: Covers damage to your vehicle from accidents
                    - Comprehensive Coverage: Non-collision damage (theft, weather, vandalism)
                    - Personal Injury Protection (PIP): Medical expenses for you and passengers
                    - Uninsured/Underinsured Motorist: Protection against drivers with insufficient coverage
                    
                    HOME INSURANCE:
                    - Dwelling Coverage: Structure of your home
                    - Personal Property: Belongings inside your home
                    - Liability Protection: Accidents on your property
                    - Additional Living Expenses (ALE): Temporary housing during repairs
                    - Medical Payments: Minor medical expenses for guests
                    
                    LIFE INSURANCE:
                    - Term Life: Fixed period coverage (10, 20, 30 years)
                    - Whole Life: Permanent coverage with cash value
                    - Universal Life: Flexible premiums and death benefits
                    - Variable Life: Investment component with market exposure
                    
                    SPECIALTY COVERAGES:
                    - Flood Insurance: Separate from standard home policies
                    - Earthquake Coverage: Requires specific endorsement
                    - Umbrella Policies: Extra liability protection
                    - Professional Liability: For service providers
                    
                    CLAIMS PROCESS:
                    1. Report incident immediately
                    2. Document damage with photos/videos
                    3. Submit required paperwork within 72 hours
                    4. Adjuster assessment within 7 business days
                    5. Settlement based on policy terms
                    
                    UNDERWRITING FACTORS:
                    - Credit history
                    - Claims history
                    - Property location
                    - Construction type
                    - Coverage limits
                    - Deductible selection
                    
                    SAMPLE POLICY FOR 25-YEAR-OLD:
                    - Term Life Insurance: $500,000 coverage for 30 years
                    - Premium: $35/month
                    - Critical Illness Rider: Additional $20/month
                    - Accidental Death Benefit: Included
                    """,
                    metadata={"source": "sample_insurance_knowledge.txt", "type": "sample"}
                )
                documents = [sample_doc]
            
            # Enhanced text splitting
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1500,
                chunk_overlap=300,
                length_function=len,
                separators=["\n\n•", "\n•", "\n\n", "\n", " ", ""]
            )
            
            split_docs = text_splitter.split_documents(documents)
            print(f" Processed {len(documents)} documents into {len(split_docs)} chunks")
            return split_docs
            
        except Exception as e:
            print(f" Error processing documents: {e}")
            return []
    
    def create_or_load_vectorstore(self, documents: List[Document] = None, force_recreate: bool = False):
        """Create or load ChromaDB vectorstore"""
        try:
            existing_collections = [col.name for col in self.chroma_client.list_collections()]
            collection_exists = self.collection_name in existing_collections
            
            if collection_exists and not force_recreate:
                print(f" Loading existing collection: {self.collection_name}")
                self.vectorstore = Chroma(
                    client=self.chroma_client,
                    collection_name=self.collection_name,
                    embedding_function=self.embeddings
                )
                collection = self.chroma_client.get_collection(self.collection_name)
                print(f" Loaded {collection.count()} documents")
                
            else:
                if collection_exists:
                    print(f"Recreating collection: {self.collection_name}")
                    self.chroma_client.delete_collection(self.collection_name)
                
                if not documents:
                    documents = self.load_and_process_documents()
                
                if not documents:
                    print(" No documents available, creating sample knowledge base")
                    documents = self.load_and_process_documents()  # Retry creating sample
                
                print(f" Creating new collection: {self.collection_name}")
                self.vectorstore = Chroma.from_documents(
                    documents=documents,
                    embedding=self.embeddings,
                    client=self.chroma_client,
                    collection_name=self.collection_name,
                    persist_directory=self.persist_directory
                )
                self.vectorstore.persist()
                print(f" Created vectorstore with {len(documents)} chunks")
            
        except Exception as e:
            print(f" Vectorstore error: {e}")
            # Create sample documents as fallback
            print(" Creating fallback sample knowledge base")
            sample_doc = Document(
                page_content="Fallback insurance knowledge base",
                metadata={"source": "fallback.txt", "type": "sample"}
            )
            self.vectorstore = Chroma.from_documents(
                documents=[sample_doc],
                embedding=self.embeddings,
                client=self.chroma_client,
                collection_name=self.collection_name,
                persist_directory=self.persist_directory
            )
    
    def retrieve_context(self, query: str, k: int = 5) -> str:
        """Retrieve relevant context with metadata"""
        try:
            if not self.vectorstore:
                self.create_or_load_vectorstore()  # Ensure vectorstore exists
                if not self.vectorstore:
                    return "Knowledge base not available"
            
            results = self.vectorstore.similarity_search_with_score(query, k=k)
            
            context_parts = []
            for i, (doc, score) in enumerate(results, 1):
                metadata = doc.metadata
                source = metadata.get("source", "Unknown document")
                page = metadata.get("page", "N/A")
                relevance = 1 - score
                
                context_parts.append(
                    f" Source {i}: {source} (Page {page}, Relevance: {relevance:.2f})\n"
                    f"{doc.page_content[:500]}{'...' if len(doc.page_content) > 500 else ''}"
                )
            
            return "\n\n".join(context_parts)
            
        except Exception as e:
            print(f" Retrieval error: {e}")
            return "Error retrieving information"
    
    def search_documents(self, query: str, k: int = 5) -> List[Dict[str, Any]]:
        """Search documents with detailed metadata"""
        try:
            if not self.vectorstore:
                return []
            
            results = self.vectorstore.similarity_search_with_score(query, k=k)
            
            search_results = []
            for doc, score in results:
                metadata = doc.metadata
                search_results.append({
                    "content": doc.page_content,
                    "metadata": metadata,
                    "relevance_score": 1 - score,
                    "source": metadata.get("source", "Unknown"),
                    "page": metadata.get("page", "N/A")
                })
            
            return search_results
            
        except Exception as e:
            print(f" Search error: {e}")
            return []
    
    def get_collection_stats(self) -> Dict[str, Any]:
        """Get ChromaDB statistics"""
        try:
            stats = {
                "persist_directory": self.persist_directory,
                "collection": self.collection_name,
                "embedding_model": AZURE_CONFIG["embedding_deployment"]
            }
            
            if self.chroma_client:
                collections = self.chroma_client.list_collections()
                stats["collections"] = [col.name for col in collections]
                
                if self.collection_name in stats["collections"]:
                    collection = self.chroma_client.get_collection(self.collection_name)
                    stats["document_count"] = collection.count()
            
            return stats
            
        except Exception as e:
            return {"error": str(e)}
    
    def reset_vectorstore(self):
        """Reset ChromaDB collection"""
        try:
            if self.chroma_client and self.collection_name in [col.name for col in self.chroma_client.list_collections()]:
                self.chroma_client.delete_collection(self.collection_name)
                print(f"Deleted collection: {self.collection_name}")
            
            self.vectorstore = None
            print(" Vectorstore reset")
            # Reinitialize after reset
            self.create_or_load_vectorstore()
            
        except Exception as e:
            print(f" Reset error: {e}")


In [ ]:


class InsuranceMultiAgentSystem:
    """Enhanced multi-agent system for insurance processing with detailed agent tracking"""
    
    def __init__(self, rag_system: InsuranceRAGSystem):
        self.rag_system = rag_system
        self.agents = {}
        self.group_chat = None
        self.manager = None
        self.setup_agents()
        self.setup_group_chat()
    
    def setup_agents(self):
        """Initialize specialized insurance agents with enhanced functionality"""
        config_list = [{
            "model": AZURE_CONFIG["gpt_deployment"],
            "api_type": "azure",
            "base_url": AZURE_CONFIG["base_url"],
            "api_key": AZURE_CONFIG["api_key"],
            "api_version": AZURE_CONFIG["api_version"]
        }]
        
        llm_config = {
            "config_list": config_list,
            "temperature": 0.1,
            "timeout": 120,
            "cache_seed": 42
        }
        
        # Knowledge Retrieval Agent
        self.agents["retriever"] = ConversableAgent(
            name="KnowledgeRetriever",
            system_message="""You are an Insurance Knowledge Specialist. Your responsibilities:
            - Retrieve accurate policy information from ChromaDB knowledge base
            - Provide citations with source documents
            - Filter irrelevant information
            - Identify knowledge gaps in the database
            - Never guess or speculate beyond available information""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=2
        )
        
        # Claims Processing Agent
        self.agents["claims_agent"] = ConversableAgent(
            name="ClaimsSpecialist",
            system_message="""You are a Senior Claims Adjuster with 10+ years experience. Responsibilities:
            - Guide customers through claims process
            - Explain documentation requirements
            - Calculate settlement estimates
            - Identify coverage limitations
            - Handle complex claims (multi-vehicle, natural disasters)
            - Provide empathetic support during stressful situations""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=3
        )
        
        # Policy Advisor Agent
        self.agents["policy_advisor"] = ConversableAgent(
            name="PolicyAdvisor",
            system_message="""You are a Licensed Policy Consultant. Responsibilities:
            - Explain coverage options and limitations
            - Recommend policy enhancements based on life changes
            - Compare insurance products across providers
            - Analyze premium/deductible tradeoffs
            - Identify coverage gaps
            - Explain riders and endorsements""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=3
        )
        
        # Customer Service Agent
        self.agents["customer_service"] = ConversableAgent(
            name="CustomerService",
            system_message="""You are the Primary Customer Interface. Responsibilities:
            - Triage inquiries to appropriate specialists
            - Maintain conversation context and history
            - Provide policy documentation
            - Handle billing inquiries
            - Ensure customer satisfaction
            - Escalate complex issues""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=2
        )
        
        # Compliance Agent
        self.agents["compliance_agent"] = ConversableAgent(
            name="ComplianceOfficer",
            system_message="""You are an Insurance Compliance Specialist. Responsibilities:
            - Ensure regulatory compliance (NAIC, state-specific)
            - Verify accurate coverage descriptions
            - Review policy limitations and exclusions
            - Monitor for unfair claims practices
            - Ensure proper disclosures
            - Maintain documentation standards""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=2
        )
        
        # Underwriting Agent
        self.agents["underwriting_agent"] = ConversableAgent(
            name="UnderwritingSpecialist",
            system_message="""You are a Senior Underwriter. Responsibilities:
            - Assess risk profiles for policy applications
            - Determine appropriate coverage levels
            - Calculate premiums based on risk assessment
            - Identify special underwriting considerations
            - Recommend policy terms and conditions""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=3
        )
        
        # Fraud Detection Agent
        self.agents["fraud_detector"] = ConversableAgent(
            name="FraudDetection",
            system_message="""You are a Fraud Detection Analyst. Responsibilities:
            - Identify suspicious claim patterns
            - Detect inconsistencies in documentation
            - Recommend investigation strategies
            - Maintain fraud detection protocols
            - Ensure compliance with anti-fraud regulations""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=2
        )
        
        # Supervisor Agent - FIXED TERMINATION ISSUE
        self.agents["supervisor"] = ConversableAgent(
            name="Supervisor",
            system_message="""You are the Team Supervisor. Responsibilities:
            - Coordinate agent responses
            - Ensure comprehensive coverage of all issues
            - Resolve inter-agent disagreements
            - Finalize responses after compliance review
            - Maintain service quality standards
            - In your final response, state which agent primarily handled the query
            - Include source references when available
            - DO NOT include the word TERMINATE in your response""",  # Removed termination trigger
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=1,
            # Removed termination condition to prevent premature ending
        )
    
    def setup_group_chat(self):
        """Configure group chat workflow"""
        agent_list = [
            self.agents["customer_service"],
            self.agents["retriever"],
            self.agents["claims_agent"],
            self.agents["policy_advisor"],
            self.agents["underwriting_agent"],
            self.agents["fraud_detector"],
            self.agents["compliance_agent"],
            self.agents["supervisor"]
        ]
        
        self.group_chat = GroupChat(
            agents=agent_list,
            messages=[],
            max_round=12,
            speaker_selection_method="auto",
            allow_repeat_speaker=False
        )
        
        manager_config = [{
            "model": AZURE_CONFIG["gpt_deployment"],
            "api_type": "azure",
            "base_url": AZURE_CONFIG["base_url"],
            "api_key": AZURE_CONFIG["api_key"],
            "api_version": AZURE_CONFIG["api_version"]
        }]
        
        self.manager = GroupChatManager(
            groupchat=self.group_chat,
            llm_config={"config_list": manager_config, "temperature": 0.1}
        )
    
    def process_query(self, query: str) -> Dict[str, Any]:
        """Process insurance query through multi-agent system with detailed agent tracking"""
        try:
            # Retrieve context before agent processing
            context = self.rag_system.retrieve_context(query)
            search_results = self.rag_system.search_documents(query, k=3)
            
            enhanced_query = f"""
            ## INSURANCE QUERY PROCESSING ##
            Customer Query: {query}
            
            Relevant Context from Knowledge Base:
            {context if context else 'No relevant context found'}
            
            Processing Instructions:
            1. KnowledgeRetriever: Verify policy details
            2. Specialist Agents: Address specific aspects
            3. ComplianceOfficer: Validate regulatory compliance
            4. Supervisor: Finalize response and state primary agent
            5. Include source references in final response
            """
            
            self.rag_system.memory.chat_memory.add_user_message(query)
            chat_result = self.agents["customer_service"].initiate_chat(
                self.manager,
                message=enhanced_query,
                clear_history=True,
                silent=True  # Set to False for debugging
            )
            
            # Extract final response and agent information
            final_response = ""
            primary_agent = "Unknown"
            all_agents = set()
            
            if chat_result.chat_history:
                # Get all unique agents involved
                all_agents = {msg["name"] for msg in chat_result.chat_history}
                
                # Extract the final response from supervisor
                for msg in reversed(chat_result.chat_history):
                    if msg["name"] == "Supervisor" and "content" in msg:
                        final_response = msg["content"]
                        break
                
                # Determine primary agent - first specialist to respond
                specialist_agents = ["ClaimsSpecialist", "PolicyAdvisor", 
                                    "UnderwritingSpecialist", "FraudDetection"]
                for msg in chat_result.chat_history:
                    if msg["name"] in specialist_agents:
                        primary_agent = msg["name"]
                        break
            
            # If supervisor didn't identify primary, try to parse from response
            if "primarily handled by" not in final_response.lower():
                # Try to determine primary agent from response content
                for agent_name in specialist_agents:
                    if agent_name.lower() in final_response.lower():
                        primary_agent = agent_name
                        break
                
                # Add primary agent declaration to response
                final_response = f"This query was primarily handled by {primary_agent}.\n\n{final_response}"
            
            self.rag_system.memory.chat_memory.add_ai_message(final_response)
            
            # Format agent list for display
            agent_list = list(all_agents)
            agent_list.sort()
            agents_involved = ", ".join(agent_list)
            
            return {
                "query": query,
                "response": final_response,
                "primary_agent": primary_agent,
                "agents_involved": agents_involved,
                "knowledge_used": bool(context.strip()),
                "search_results": search_results
            }
            
        except Exception as e:
            print(f" Processing error: {e}")
            return {
                "query": query,
                "response": f"System error: {str(e)}",
                "primary_agent": "Error",
                "agents_involved": "",
                "knowledge_used": False,
                "search_results": []
            }
    
    def simple_query(self, query: str) -> Dict[str, Any]:
        """Direct RAG response for simple queries with agent simulation"""
        try:
            context = self.rag_system.retrieve_context(query)
            search_results = self.rag_system.search_documents(query, k=3)
            
            prompt = f"""
            [Insurance Expert Mode]
            Answer the customer's question using ONLY the provided context.
            Include source references when available.
            
            Question: {query}
            
            Context:
            {context if context else 'No relevant information available'}
            
            Instructions:
            1. Answer concisely (1-2 paragraphs)
            2. Cite sources when available
            3. If context is insufficient, state: "Based on my knowledge: [answer]. Consult your policy for specifics."
            4. Never speculate beyond provided information
            """
            
            response = self.rag_system.llm.invoke([HumanMessage(content=prompt)])
            
            # Simulate agent involvement for simple queries
            return {
                "query": query,
                "response": response.content,
                "primary_agent": "KnowledgeRetriever",
                "agents_involved": "KnowledgeRetriever, CustomerService",
                "knowledge_used": bool(context.strip()),
                "search_results": search_results
            }
            
        except Exception as e:
            print(f"⚠️ Simple query error: {e}")
            return {
                "query": query,
                "response": f"System error: {str(e)}",
                "primary_agent": "Error",
                "agents_involved": "",
                "knowledge_used": False,
                "search_results": []
            }


In [ ]:

def main():
    """Main application flow with enhanced agent reporting"""
    print("\n" + "=" * 70)
    print("INSURANCE MULTI-AGENT RAG SYSTEM WITH CHROMADB".center(70))
    print("=" * 70)
    
    try:
        # Initialize RAG system
        print("\n🔧 Initializing RAG system...")
        rag_system = InsuranceRAGSystem(
            data_folder="data",
            persist_directory="chroma_db"
        )
        
        # Check existing vectorstore
        stats = rag_system.get_collection_stats()
        doc_count = stats.get("document_count", 0)
        print(f" ChromaDB Status: {doc_count} documents in '{stats.get('collection', '')}'")
        
        if doc_count < 10:
            print("\n Loading documents...")
            rag_system.create_or_load_vectorstore()
            stats = rag_system.get_collection_stats()
            print(f"New Status: {stats.get('document_count', 0)} documents")
        
        # Initialize agent system
        print("\n Initializing multi-agent system...")
        agent_system = InsuranceMultiAgentSystem(rag_system)
        print(" System ready with 8 specialized agents")
        
        # Interactive mode
        print("\n" + "=" * 70)
        print(" INTERACTIVE MODE - Ask insurance questions".center(70))
        print(" Commands: stats, search [query], reload, reset, quit".center(70))
        print("=" * 70)
        
        while True:
            try:
                user_input = input("\n Your question: ").strip()
                
                if not user_input:
                    continue
                    
                if user_input.lower() in ['quit', 'exit', 'q']:
                    print("\n Thank you for using the Insurance Assistant!")
                    break
                    
                # Handle special commands
                if user_input.lower() == 'stats':
                    stats = rag_system.get_collection_stats()
                    print("\n ChromaDB Statistics:")
                    for key, value in stats.items():
                        print(f"  - {key}: {value}")
                    continue
                    
                if user_input.lower().startswith('search '):
                    query = user_input[7:].strip()
                    print(f"\n Searching for: '{query}'")
                    results = rag_system.search_documents(query, k=3)
                    
                    if results:
                        for i, result in enumerate(results, 1):
                            print(f"\n Result {i}:")
                            print(f"   Source: {result.get('source', 'Unknown')}")
                            print(f"   Page: {result.get('page', 'N/A')}")
                            print(f"   Relevance: {result.get('relevance_score', 0):.3f}")
                            print(f"   Content: {result['content'][:150]}...")
                    else:
                        print(" No results found")
                    continue
                    
                if user_input.lower() == 'reload':
                    print("\n Reloading documents...")
                    rag_system.create_or_load_vectorstore(force_recreate=True)
                    print(" Documents reloaded successfully!")
                    continue
                    
                if user_input.lower() == 'reset':
                    confirm = input(" Delete ALL documents? (yes/no): ")
                    if confirm.lower() in ['yes', 'y']:
                        rag_system.reset_vectorstore()
                        print(" ChromaDB reset complete")
                    continue
                
                # Process query
                start_time = time.time()
                print("\n Processing your query with our agent team...")
                
                # Use multi-agent for complex queries, simple for others
                if "claim" in user_input.lower() or "policy" in user_input.lower() or "coverage" in user_input.lower():
                    result = agent_system.process_query(user_input)
                else:
                    result = agent_system.simple_query(user_input)
                
                # Display results with agent information
                print("\n" + "-" * 70)
                print(f" Query: {user_input}")
                print(f" Primarily handled by: {result['primary_agent']}")
                print(f" Agents involved: {result['agents_involved']}")
                print(f" Knowledge base was consulted: {'Yes' if result['knowledge_used'] else 'No'}")
                print("-" * 70)
                
                # Display response
                print("\n Response:")
                print("-" * 70)
                print(result["response"])
                print("-" * 70)
                print(f"⏱  Response time: {time.time() - start_time:.2f} seconds")
                
                # Show relevant sources if available
                if result.get("search_results"):
                    print("\n Relevant Sources:")
                    for i, res in enumerate(result["search_results"][:3], 1):
                        source = res.get('source', 'Unknown')
                        page = res.get('page', 'N/A')
                        relevance = res.get('relevance_score', 0)
                        print(f"  {i}. {source} (Page {page}, Relevance: {relevance:.2f})")
                elif result["knowledge_used"]:
                    print("\n Knowledge base was consulted but no specific sources are available")
                
            except KeyboardInterrupt:
                print("\n Operation cancelled")
            except Exception as e:
                print(f"\n Error: {str(e)}")
    
    except Exception as e:
        print(f"\n Fatal error: {e}")
        import traceback
        traceback.print_exc()

In [6]:




if __name__ == "__main__":
    main()


            INSURANCE MULTI-AGENT RAG SYSTEM WITH CHROMADB            

🔧 Initializing RAG system...


C:\Users\dhanu\AppData\Local\Temp\ipykernel_23336\3574312706.py:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(


✅ Azure OpenAI clients initialized successfully
🔵 ChromaDB client initialized at: chroma_db
♻️ Loading existing collection: insurance_documents


C:\Users\dhanu\AppData\Local\Temp\ipykernel_23336\3574312706.py:187: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vectorstore = Chroma(


📚 Loaded 316 documents
📊 ChromaDB Status: 316 documents in 'insurance_documents'

🤖 Initializing multi-agent system...
✅ System ready with 8 specialized agents

             💬 INTERACTIVE MODE - Ask insurance questions             
        🔧 Commands: stats, search [query], reload, reset, quit        

⏳ Processing your query with our agent team...

----------------------------------------------------------------------
🧑‍💼 Query: suggest me some HLA policies
🤖 Primarily handled by: KnowledgeRetriever
🤖 Agents involved: KnowledgeRetriever, CustomerService
📚 Knowledge base was consulted: Yes
----------------------------------------------------------------------

💡 Response:
----------------------------------------------------------------------
Based on the provided context, here are two HLA policies you may consider:

1. **HLA CompleteProtect**: This policy offers enhanced protection to help you embrace life's uncertainties. It is underwritten by Hong Leong Assurance Berhad and regulated